In [1]:
# ===========================================
# 3) Model Building — Upwork Jobs 
# ===========================================
# Bu bölümde CSV verisinden makine öğrenmesi modelleri oluşturuluyor.
# Amaç: Upwork ilanlarının türünü ve ücretini tahmin edebilmek.

import os, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    r2_score, mean_absolute_error, root_mean_squared_error
)
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


In [2]:
# ---------- 1) Tek CSV Dosyasını Yükleme & Basit Ön İşleme ----------
DATA_DIR = r"C:\Users\karm1\OneDrive\Desktop\Upwork_Project\data"
CSV_FILE = "all_upwork_jobs_2024-02-07-2024-03-24.csv"  # Dosya adını değiştirdiysen burada güncelle
csv_path = os.path.join(DATA_DIR, CSV_FILE)

# Dosya mevcut mu kontrol et
if not os.path.isfile(csv_path):
    raise FileNotFoundError(f"CSV not found at: {csv_path}")


In [3]:
# CSV dosyasını oku
df = pd.read_csv(csv_path, low_memory=False)
print("Loaded shape:", df.shape)
print("Columns:", list(df.columns))


Loaded shape: (244828, 8)
Columns: ['title', 'link', 'published_date', 'is_hourly', 'hourly_low', 'hourly_high', 'budget', 'country']


In [4]:
# Eğer "hourly_low" ve "hourly_high" varsa ortalamasını alarak yeni "hourly_rate" sütunu oluştur
if {"hourly_low","hourly_high"}.issubset(df.columns):
    df["hourly_rate"] = (pd.to_numeric(df["hourly_low"], errors="coerce") +
                         pd.to_numeric(df["hourly_high"], errors="coerce")) / 2


In [5]:
# "is_hourly" sütununu 0 ve 1 formatına dönüştür (yani sabit mi, saatlik mi?)
if "is_hourly" in df.columns:
    def map_hourly(v):
        s = str(v).strip().lower()
        # Aşağıdaki değerler 1 (saatlik iş) olarak değerlendirilir
        if s in ["1","true","yes","y","hourly","hourly_job","t"]: return 1
        # Aşağıdaki değerler 0 (sabit ücretli iş) olarak değerlendirilir
        if s in ["0","false","no","n","fixed","fixed_price","f"]: return 0
        # Diğer durumlarda dönüştürme yapılamaz → NaN
        try: return int(v)
        except: return np.nan
    df["is_hourly"] = df["is_hourly"].apply(map_hourly)


In [6]:
# Sayısal sütunlardaki anormal veya negatif değerleri temizle
for c in ["budget","hourly_rate"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
        if df[c].notna().any():
            q99 = df[c].quantile(0.99)  # %99’luk eşik değeri
            df.loc[df[c] < 0, c] = np.nan
            df.loc[df[c] > q99*5, c] = np.nan  # Çok büyük outlier’ları kaldır


In [7]:
# Metin ve kategorik sütunları standart hale getir
df["title"] = df.get("title","").astype(str).fillna("")
df["country"] = df.get("country","unknown").astype(str).fillna("unknown")


In [8]:
# Büyük veri setlerinde işlem hızını artırmak için örnekleme (örneğin 245k satır → 50k satır)
SAMPLE_MAX = 50000  # Bilgisayarın yavaşsa 30000 yapabilirsin
df_sample = df.sample(SAMPLE_MAX, random_state=42) if len(df) > SAMPLE_MAX else df.copy()
print("Working sample shape:", df_sample.shape)


Working sample shape: (50000, 9)


In [9]:
# Ortak ön işleme fonksiyonu: TF-IDF(title) + OneHot(country)
def make_preprocessor():
    return ColumnTransformer(
        transformers=[
            # TF-IDF ile ilan başlığındaki kelimeleri sayısal vektörlere dönüştürür
            ("txt", TfidfVectorizer(max_features=4000, ngram_range=(1,2)), "title"),  # 4000 özellik yeterli hız sağlar
            # Ülke bilgisini One-Hot Encoder ile sayısallaştırır
            ("cty", OneHotEncoder(handle_unknown="ignore"), ["country"]),
        ],
        remainder="drop",
        verbose_feature_names_out=False
    )

# Yardımcı fonksiyon: sadece gerekli sütunları alır
def build_X(dfin):
    return pd.DataFrame({
        "title": dfin["title"].astype(str),
        "country": dfin["country"].astype(str)
    })


In [10]:
# ---------- 2) Sınıflandırma (Classification): 'is_hourly' tahmini ----------
# Amaç: İlan saatlik mi (1), sabit ücretli mi (0)?
if "is_hourly" in df_sample.columns and df_sample["is_hourly"].dropna().nunique()==2:
    dcls = df_sample.dropna(subset=["is_hourly"]).copy()
    y = dcls["is_hourly"].astype(int)  # hedef değişken
    X = build_X(dcls)                  # giriş özellikleri

    pre = make_preprocessor()
    # Kullanılacak modeller:
    models_cls = {
        "LogReg": LogisticRegression(max_iter=1000, solver="liblinear"),  # Basit doğrusal model
        "RFCls": RandomForestClassifier(n_estimators=250, random_state=42, n_jobs=-1),  # Güçlü ağaç modeli
    }

    # Veriyi eğitim (%80) ve test (%20) olarak ayır
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print("\n[Classification] Train:", X_train.shape, "Test:", X_test.shape)

    # Her model için eğitim ve değerlendirme
    cls_rows = []
    for name, model in models_cls.items():
        pipe = Pipeline([("prep", pre), ("model", model)])  # Ön işleme + model zinciri
        pipe.fit(X_train, y_train)  # Modeli eğit
        y_pred = pipe.predict(X_test)  # Tahmin yap
        acc = accuracy_score(y_test, y_pred)  # Doğruluk oranı
        f1 = f1_score(y_test, y_pred)         # F1 skoru
        cls_rows.append({"Model": name, "ACC": acc, "F1": f1})
        print(f"[CLS] {name}: ACC={acc:.4f}  F1={f1:.4f}")
        print(classification_report(y_test, y_pred, digits=3))
    # Sonuç tablosu
    cls_results = pd.DataFrame(cls_rows).sort_values("F1", ascending=False).reset_index(drop=True)
    print("\n[CLS] Summary:\n", cls_results)
else:
    print("\n[Classification] Skipped: 'is_hourly' not available or not binary.")



[Classification] Train: (40000, 2) Test: (10000, 2)
[CLS] LogReg: ACC=0.6536  F1=0.7197
              precision    recall  f1-score   support

           0      0.612     0.494     0.547      4228
           1      0.675     0.770     0.720      5772

    accuracy                          0.654     10000
   macro avg      0.644     0.632     0.633     10000
weighted avg      0.648     0.654     0.647     10000

[CLS] RFCls: ACC=0.6509  F1=0.7182
              precision    recall  f1-score   support

           0      0.609     0.487     0.541      4228
           1      0.672     0.771     0.718      5772

    accuracy                          0.651     10000
   macro avg      0.641     0.629     0.630     10000
weighted avg      0.646     0.651     0.643     10000


[CLS] Summary:
     Model     ACC        F1
0  LogReg  0.6536  0.719696
1   RFCls  0.6509  0.718218


In [11]:
# ---------- 3) Regresyon A: Sabit işler → 'budget' tahmini ----------
# Amaç: Sabit ücretli işlerin proje bütçesini tahmin etmek.
regA_done = False
if {"is_hourly","budget"}.issubset(df_sample.columns):
    dreg = df_sample[(df_sample["is_hourly"]==0) & (df_sample["budget"].notna())].copy()
    if len(dreg) >= 200:  # Yeterli veri olmalı
        y = dreg["budget"].astype(float)
        X = build_X(dreg)
        pre = make_preprocessor()
        # İki model denenecek: Doğrusal Regresyon ve Random Forest
        models_reg = {
            "LinReg": LinearRegression(),
            "RFReg": RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1),
        }
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        print(f"\n[Regression-Fixed] Rows={len(dreg)}  Train:{X_train.shape}  Test:{X_test.shape}")
        reg_rows = []
        for name, model in models_reg.items():
            pipe = Pipeline([("prep", pre), ("model", model)])
            pipe.fit(X_train, y_train)
            y_pred = pipe.predict(X_test)
            # Performans metrikleri:
            r2  = r2_score(y_test, y_pred)  # Modelin açıklama gücü (0–1 arası)
            mae = mean_absolute_error(y_test, y_pred)  # Ortalama hata miktarı
            rmse= root_mean_squared_error(y_test, y_pred)  # Kök ortalama kare hata
            reg_rows.append({"Model": name, "R2": r2, "MAE": mae, "RMSE": rmse})
            print(f"[REG-Fixed] {name}: R2={r2:.4f}  MAE={mae:.2f}  RMSE={rmse:.2f}")
        # Sonuçları tablo halinde yazdır
        regA_results = pd.DataFrame(reg_rows).sort_values("RMSE").reset_index(drop=True)
        print("\n[REG-Fixed] Summary:\n", regA_results)
        regA_done = True

if not regA_done:
    print("\n[Regression-Fixed] Skipped: need 'budget' with is_hourly==0 rows.")



[Regression-Fixed] Rows=21104  Train:(16883, 2)  Test:(4221, 2)
[REG-Fixed] LinReg: R2=-0.1673  MAE=964.72  RMSE=2548.49
[REG-Fixed] RFReg: R2=-0.1051  MAE=616.83  RMSE=2479.63

[REG-Fixed] Summary:
     Model        R2         MAE         RMSE
0   RFReg -0.105051  616.826112  2479.628584
1  LinReg -0.167275  964.715809  2548.485346


In [12]:
# ---------- 4) Regresyon B: Saatlik işler → 'hourly_rate' tahmini ----------
# Amaç: Saatlik işlerin ortalama saatlik ücretini tahmin etmek.
regB_done = False
if {"is_hourly","hourly_rate"}.issubset(df_sample.columns):
    dreg_h = df_sample[(df_sample["is_hourly"]==1) & (df_sample["hourly_rate"].notna())].copy()
    if len(dreg_h) >= 200:
        y = dreg_h["hourly_rate"].astype(float)
        X = build_X(dreg_h)
        pre = make_preprocessor()
        models_reg = {
            "LinReg": LinearRegression(),
            "RFReg": RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1),
        }
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        print(f"\n[Regression-Hourly] Rows={len(dreg_h)}  Train:{X_train.shape}  Test:{X_test.shape}")
        reg_rows = []
        for name, model in models_reg.items():
            pipe = Pipeline([("prep", pre), ("model", model)])
            pipe.fit(X_train, y_train)
            y_pred = pipe.predict(X_test)
            r2  = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse= root_mean_squared_error(y_test, y_pred)
            reg_rows.append({"Model": name, "R2": r2, "MAE": mae, "RMSE": rmse})
            print(f"[REG-Hourly] {name}: R2={r2:.4f}  MAE={mae:.2f}  RMSE={rmse:.2f}")
        regB_results = pd.DataFrame(reg_rows).sort_values("RMSE").reset_index(drop=True)
        print("\n[REG-Hourly] Summary:\n", regB_results)
        regB_done = True

if not regB_done:
    print("\n[Regression-Hourly] Skipped: need 'hourly_rate' with is_hourly==1 rows.")



[Regression-Hourly] Rows=20253  Train:(16202, 2)  Test:(4051, 2)
[REG-Hourly] LinReg: R2=0.0043  MAE=17.91  RMSE=28.57
[REG-Hourly] RFReg: R2=0.1038  MAE=14.93  RMSE=27.10

[REG-Hourly] Summary:
     Model        R2        MAE       RMSE
0   RFReg  0.103837  14.928130  27.103331
1  LinReg  0.004334  17.905503  28.568406


In [15]:
print("\nDone. işlem tamamlandı")  # Tüm işlemler tamamlandı



Done. işlem tamamlandı
